In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.surf_scrape_db
collection = db.surf_summary

In [4]:
# URL of page to be scraped
url = 'https://www.surfline.com/surf-reports-forecasts-cams/costa-rica/362406'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, "html.parser")
# soup = BeautifulSoup(html, "html.parser")
# soup = BeautifulSoup(response.text, 'lxml')


In [5]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
results = soup.find_all("h3", class_="sl-spot-details__name")

# Loop through returned results
for result in results:
    # Error handling
    try:
        # Identify and return location 
        location = result.find("h3", class_="sl-spot-details__name").get_text()
        
        # Identify and return quiver surf height
        quiver_surf_height = result.find("span", class_="quiver-surf-height").get_text()
        
        # Identify and return link to listing
        url_link = result.find("a", class_="sl-cam-list-link", href=True)

        # Run only if title, price, and link are available
        if (location and quiver_surf_height and url_link):
            # Print results
            print('-------------')
            print(location)
            print(quiver_surf_height)
            print(url_link)

            # Dictionary to be inserted as a MongoDB document
            post = {
                'location': location,
                'quiver_surf_height': quiver_surf_height,
                'url': url_link
            }

            collection.insert_one(post)

    except Exception as e:
        print(e)  

In [7]:
# Display items in MongoDB collection
listings = db.surf_summary.find()

for listing in listings:
    print(listing)